In [1]:
import os,sys
pwd = os.getcwd()
parent = '/'.join(pwd.split('/')[:-
                                 1])
sys.path.insert(0,parent)
#%%
print('-'*30)
print(os.getcwd())
print('-'*30)
#%%
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from core.DAZLE import DAZLE
from core.SUNDataLoader import SUNDataLoader
from core.helper_func import eval_zs_gzsl,get_lr#,get_attribute_attention_stats,
from global_setting import NFS_path

------------------------------
/home/project_amadeus/home/hbdat/[RELEASE]_DenseAttentionZSL/notebook
------------------------------


In [2]:
idx_GPU = 2
device = torch.device("cuda:{}".format(idx_GPU) if torch.cuda.is_available() else "cpu")

In [3]:
torch.backends.cudnn.benchmark = True

In [4]:
dataloader = SUNDataLoader(NFS_path,device,is_scale=False,is_balance = True)

/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
SUN
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/data/SUN/feature_map_ResNet_101_SUN.hdf5
Expert Attr
Finish loading data in  80.897228


In [ ]:
#%%
seed = 214
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

print('Randomize seed {}'.format(seed))
#%%
batch_size = 50
nepoches = 100
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att
att = dataloader.att#dataloader.normalize_att#
normalize_att = dataloader.normalize_att
#assert (att.min().item() == 0 and att.max().item() == 1)

trainable_w2v = True
lambda_ = 0.1
bias = 0.
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = True

seenclass = dataloader.seenclasses
unseenclass = dataloader.unseenclasses
desired_mass = 1#unseenclass.size(0)/(seenclass.size(0)+unseenclass.size(0))
report_interval = niters//nepoches
#%%
model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=False,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True,non_linear_act=False)
model.to(device)
#%%
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.0001#0.000#0.#
momentum = 0.9#0.#
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)
#%%
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

In [6]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss,loss_CE,loss_cal = out_package['loss'],out_package['loss_CE'],out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen -0.0 bias_unseen 0.0
{'iter': 0, 'loss': 6.891836166381836, 'loss_CE': 6.811451435089111, 'loss_cal': 0.8038462400436401, 'acc_seen': 0, 'acc_novel': 0, 'H': 0, 'acc_zs': 0}
------------------------------
bias_seen -0.0 bias_unseen 0.0
{'iter': 206, 'loss': 3.6871535778045654, 'loss_CE': 3.5833122730255127, 'loss_cal': 1.0384119749069214, 'acc_seen': 0.08139535039663315, 'acc_novel': 0.526388943195343, 'H': 0.14098953503086709, 'acc_zs': 0.5479167103767395}
------------------------------
bias_seen -0.0 bias_unseen 0.0
{'iter': 412, 'loss': 3.001089572906494, 'loss_CE': 2.893244504928589, 'loss_cal': 1.0784509181976318, 'acc_seen': 0.1236434131860733, 'acc_novel': 0.5444444417953491, 'H': 0.20152130762992324, 'acc_zs': 0.5756944417953491}
------------------------------
bias_seen -0.0 bias_unseen 0.0
{'iter': 618, 'loss': 3.061366558074951, 'loss_CE': 2.9667210578918457, 'loss_cal': 0.9464538097381592, 'acc_seen': 0.1523255854845047, 'acc_novel': 